In [ ]:
import pandas as pd
import numpy as np
import datetime
from datetime import datetime, timedelta
import seaborn as sns
import matplotlib.pyplot as plt

import time

import warnings
warnings.simplefilter(action = 'ignore', category = FutureWarning)

# imports best practice pandas
import os

import numpy as np
import pandas as pd

#--------------------------------------------------------
# imports best practice sklearn
import sklearn
from sklearn import set_config

from sklearn.tree import DecisionTreeClassifier


# transformers
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OrdinalEncoder, OneHotEncoder

# pipelines
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

# model selection e hiperparameters optimization
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, RepeatedKFold, KFold

from sklearn.metrics import accuracy_score


### Load sales and prices datasets, both without nulls and duplicates

In [ ]:
df_sales = pd.read_csv('./data/reduced_sales.csv')

In [ ]:
df_prices = pd.read_csv('./data/reduced_prices.csv')


In [ ]:
del(df_sales["Unnamed: 0"])

In [ ]:
df_sales

In [ ]:
del(df_prices["Unnamed: 0"])

In [ ]:
df_prices

### Making new colunm so to merge by it

In [ ]:
df_prices["id"] = df_prices["item"] + "_" + df_prices["store_code"]

In [ ]:
df_prices

### deleting useless colunms

In [ ]:
del(df_prices["item"])

In [ ]:
del(df_prices["store_code"])

In [ ]:
df_prices

### Deleting days on df["dates]

In [ ]:
df_prices["date"] = df_prices["date"].astype(str).apply(lambda x: x[0:7])

In [ ]:
df_prices.shape

In [ ]:
df_prices.head(5)

In [ ]:
df_prices

In [ ]:
df_sales

In [ ]:
df_full = pd.merge(df_sales,df_prices, on=["id", "date"], how="left")

### Filling nulls with 0

In [ ]:
df_full["sell_price"].fillna(0, inplace=True)

###  Making new columns "Income"

In [ ]:
df_full["income"] = df_full["sales"] * df_full["sell_price"]

### Loading original item_sales_df for making dicts to map new columns

In [ ]:
df_full_sales = pd.read_csv("./data/item_sales.csv")

In [ ]:
df_full_sales

In [ ]:
store_dict = df_full_sales.groupby(["store_code"])["store"].first().to_dict()

In [ ]:
region_dict = df_full_sales.groupby(["store_code"])["region"].first().to_dict()

### Making new columns from "id" column

In [ ]:
df_full["store_code"] = df_full["id"].apply(lambda x: x[-5:])

In [ ]:
store_dict

In [ ]:
df_full["store"] = df_full["store_code"].map(store_dict)

In [ ]:
df_full["region"] = df_full["store_code"].map(region_dict)

In [ ]:
df_full["department"] = df_full["id"].apply(lambda x: x[:-12])

In [ ]:
df_full["category"] = df_full["id"].apply(lambda x: x[:-10])

In [ ]:
df_full

In [ ]:
file_name = "df_full.csv"
df_full.to_csv(file_name)